In [2]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import geopy.distance
from langdetect import detect, DetectorFactory
import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# old functions

def fuzzy_match(
    df
    ):
    '''
    add function description here
    '''
    df = df.fillna('')
    for i in df.index:
        df.loc[i,'name_ratio'] = fuzz.ratio(df.loc[i,'name_1'],df.loc[i,'name_2'])
        df.loc[i,'city_ratio'] = fuzz.ratio(df.loc[i,'city_1'],df.loc[i,'city_2'])
        df.loc[i,'state_ratio'] = fuzz.ratio(df.loc[i,'state_1'],df.loc[i,'state_2'])
        df.loc[i,'zip_ratio'] = fuzz.ratio(df.loc[i,'zip_1'],df.loc[i,'zip_2'])
        df.loc[i,'country_ratio'] = fuzz.ratio(df.loc[i,'country_1'],df.loc[i,'country_2'])
        df.loc[i,'url_ratio'] = fuzz.ratio(df.loc[i,'url_1'],df.loc[i,'url_2'])
        df.loc[i,'categories_ratio'] = fuzz.ratio(df.loc[i,'categories_1'],df.loc[i,'categories_2'])
    
        df.loc[i,'name_ratio_part'] = fuzz.partial_ratio(df.loc[i,'name_1'],df.loc[i,'name_2'])
        df.loc[i,'city_ratio_part'] = fuzz.partial_ratio(df.loc[i,'city_1'],df.loc[i,'city_2'])
        df.loc[i,'state_ratio_part'] = fuzz.partial_ratio(df.loc[i,'state_1'],df.loc[i,'state_2'])
        df.loc[i,'zip_ratio_part'] = fuzz.partial_ratio(df.loc[i,'zip_1'],df.loc[i,'zip_2'])
        df.loc[i,'country_ratio_part'] = fuzz.partial_ratio(df.loc[i,'country_1'],df.loc[i,'country_2'])
        df.loc[i,'url_ratio_part'] = fuzz.partial_ratio(df.loc[i,'url_1'],df.loc[i,'url_2'])
        df.loc[i,'categories_ratio_part'] = fuzz.partial_ratio(df.loc[i,'categories_1'],df.loc[i,'categories_2'])
    return df


def point_dist(
    df
    ):
    '''
    add function description here
    '''
    for i in df.index:
        coord_1 = (df.loc[i,'latitude_1'],df.loc[i,'longitude_1'])
        
        coord_2 = (df.loc[i,'latitude_2'],df.loc[i,'longitude_2'])
        
        df['proximity'] = geopy.distance.geodesic(coord_1,coord_2).km
    return df

def same_lang(
    df
    ):
    DetectorFactory.seed = 0
    df['same_lang'] = 0
    df = df.fillna('')
    
    for i in df.index:
        full_address_1 = detect(df.loc[i,'name_1'] + ' ' + df.loc[i,'address_1'] + ' ' + df.loc[i,'city_1'] + ' ' + df.loc[i,'state_1'] + ' ' + df.loc[i,'country_1'])
        full_address_2 = detect(df.loc[i,'name_2'] + ' ' + df.loc[i,'address_2'] + ' ' + df.loc[i,'city_2'] + ' ' + df.loc[i,'state_2'] + ' ' + df.loc[i,'country_2'])
        if (full_address_1 == full_address_2):
            df.loc[i,'same_lang'] = 1
            df.loc[i,'lang_addr_1'] = full_address_1
            df.loc[i,'lang_addr_2'] = full_address_2
            
    
    return df

def len_strings(
        df
        ):
    df = df.fillna('')
    df['len_name_1'] = df['name_1'].str.len()
    df['len_name_2'] = df['name_2'].str.len()
    df['len_address_1'] = df['address_1'].str.len()
    df['len_address_2'] = df['address_2'].str.len()
    df['len_city_1'] = df['city_1'].str.len()
    df['len_city_2'] = df['city_2'].str.len()
    df['len_state_1'] = df['state_1'].str.len()
    df['len_state_2'] = df['state_2'].str.len()
    df['len_zip_1'] = df['zip_1'].str.len()
    df['len_zip_2'] = df['zip_2'].str.len()
    df['len_phone_1'] = df['phone_1'].str.len()
    df['len_phone_2'] = df['phone_2'].str.len()
    return df

def build_features(
        df
        ):
    org_columns = df.columns
    org_columns = org_columns.drop(['id_1','id_2','MATCH'])
    df = fuzzy_match(df)
    df = point_dist(df)
    df = same_lang(df)
    df = len_strings(df)
    
    df = df.drop(org_columns, axis = 1)
    return df

In [10]:
# new functions

def detect_(
    x,
    i
):
    
    try:
        op = detect(str(x[f'name_{i}']).replace(str(np.nan), '') + ' ' + str(x[f'address_{i}']).replace(str(np.nan), '') + ' ' + str(x[f'city_{i}']).replace(str(np.nan), '') + ' ' + str(x[f'state_{i}']).replace(str(np.nan), '') + ' ' + str(x[f'country_{i}']).replace(str(np.nan), ''))
    except:
        op = np.nan
    
    return op
    

def fuzzy_match(
    df
):
    
    df['name_ratio'] = df.apply(lambda x: fuzz.ratio(x['name_1'], x['name_2']) if( (pd.isnull(x['name_1'])==False) & (pd.isnull(x['name_2'])==False) ) else np.nan, axis=1)
    df['city_ratio'] = df.apply(lambda x: fuzz.ratio(x['city_1'], x['city_2']) if( (pd.isnull(x['city_1'])==False) & (pd.isnull(x['city_2'])==False) ) else np.nan, axis=1)
    df['state_ratio'] = df.apply(lambda x: fuzz.ratio(x['state_1'], x['state_2']) if( (pd.isnull(x['state_1'])==False) & (pd.isnull(x['state_2'])==False) ) else np.nan, axis=1)
    df['zip_ratio'] = df.apply(lambda x: fuzz.ratio(x['zip_1'], x['zip_2']) if( (pd.isnull(x['zip_1'])==False) & (pd.isnull(x['zip_2'])==False) ) else np.nan, axis=1)
    df['country_ratio'] = df.apply(lambda x: fuzz.ratio(x['country_1'], x['country_2']) if( (pd.isnull(x['country_1'])==False) & (pd.isnull(x['country_2'])==False) ) else np.nan, axis=1)
    df['url_ratio'] = df.apply(lambda x: fuzz.ratio(x['url_1'], x['url_2']) if( (pd.isnull(x['url_1'])==False) & (pd.isnull(x['url_2'])==False) ) else np.nan, axis=1)
    df['categories_ratio'] = df.apply(lambda x: fuzz.ratio(x['categories_1'], x['categories_2']) if( (pd.isnull(x['categories_1'])==False) & (pd.isnull(x['categories_2'])==False) ) else np.nan, axis=1)
    
    df['name_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['name_1'], x['name_2']) if( (pd.isnull(x['name_1'])==False) & (pd.isnull(x['name_2'])==False) ) else np.nan, axis=1)
    df['city_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['city_1'], x['city_2']) if( (pd.isnull(x['city_1'])==False) & (pd.isnull(x['city_2'])==False) ) else np.nan, axis=1)
    df['state_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['state_1'], x['state_2']) if( (pd.isnull(x['state_1'])==False) & (pd.isnull(x['state_2'])==False) ) else np.nan, axis=1)
    df['zip_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['zip_1'], x['zip_2']) if( (pd.isnull(x['zip_1'])==False) & (pd.isnull(x['zip_2'])==False) ) else np.nan, axis=1)
    df['country_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['country_1'], x['country_2']) if( (pd.isnull(x['country_1'])==False) & (pd.isnull(x['country_2'])==False) ) else np.nan, axis=1)
    df['url_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['url_1'], x['url_2']) if( (pd.isnull(x['url_1'])==False) & (pd.isnull(x['url_2'])==False) ) else np.nan, axis=1)
    df['categories_ratio_part'] = df.apply(lambda x: fuzz.partial_ratio(x['categories_1'], x['categories_2']) if( (pd.isnull(x['categories_1'])==False) & (pd.isnull(x['categories_2'])==False) ) else np.nan, axis=1)
    
    return df


def point_dist(
    df
):
    
    df['proximity'] = df.apply(lambda x: geopy.distance.geodesic((x['latitude_1'],x['longitude_1']), (x['latitude_2'],x['longitude_2'])).km, axis=1)
    
    return df


def same_lang(
    df
):
    
    DetectorFactory.seed = 0
    
    df['full_address_1_lang'] = df.apply(lambda x: detect_(x, 1), axis=1)
    df['full_address_2_lang'] = df.apply(lambda x: detect_(x, 2), axis=1)
    df['same_lang'] = np.where((df['full_address_1_lang'].isna()) | (df['full_address_2_lang'].isna()),
                               np.nan,
                               np.where(df['full_address_1_lang'] == df['full_address_2_lang'], 1, 0))
    
    return df


def len_strings(
    df
):
    
    df['len_name_diff'] = df['name_1'].str.len() - df['name_2'].str.len()
    df['len_address_diff'] = df['address_1'].str.len() - df['address_2'].str.len()
    df['len_city_diff'] = df['city_1'].str.len() - df['city_2'].str.len()
    df['len_state_diff'] = df['state_1'].str.len() - df['state_2'].str.len()
    df['len_zip_diff'] = df['zip_1'].str.len() - df['zip_2'].str.len()
    df['len_phone_diff'] = df['phone_1'].str.len() - df['phone_2'].str.len()
    df['len_categories_diff'] = df['categories_1'].str.len() - df['categories_2'].str.len()
    
    return df

def build_features(
    df
):
    
    org_columns = df.columns
    org_columns = org_columns.drop(['id_1','id_2','match'])
    
    df = fuzzy_match(df)
    print('fuzzy match complete @ {}'.format(datetime.datetime.now()))
        
    df = point_dist(df)
    print('point dist complete @ {}'.format(datetime.datetime.now()))
    
    df = same_lang(df)
    print('same lang complete @ {}'.format(datetime.datetime.now()))
    
    df = len_strings(df)
    print('len strings complete @ {}'.format(datetime.datetime.now()))
    
    df = df.drop(org_columns, axis = 1)
    
    return df

In [4]:
df_raw = pd.read_csv(r'C:\Users\caseyrya\Dropbox\foursquare_location_matching_data\data\dummy_dev_data\train_pairs.csv')

print(df_raw.shape)
df_raw.head()

(1027558, 26)


,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,phone_1,categories_1,index,id_2,name_2,latitude_2,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_000001272c6c5d,café stad oudenaarde,50.859975,3.634196,abdijstraat,nederename,oostvlaanderen,9700,be,NaN,NaN,bars,634346,E_9e4a5adfe5b1c5,frituur de groten honger,50.858896,3.633336,abdijstraat 83,nederename,oostvlaanderen,NaN,be,NaN,+3255309661,friteries,False
1,E_000002eae2a589,carioca manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,br,NaN,NaN,brazilian restaurants,774258,E_c12fb799a8a0e4,claro,-22.907135,-43.178162,shopping avenida central,rio de janeiro,rj,20040901,br,httpwwwclarocombr,NaN,mobile phone shops,False
2,E_000007f24ebc95,รานตดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,th,NaN,NaN,salons barbershops,731040,E_b664578fe0e8c2,รานพนา,13.780847,100.484630,NaN,NaN,NaN,NaN,th,NaN,NaN,nail salons,False
3,E_000008a8ba4f48,turkcell,37.844510,27.844202,adnan menderes bulvarı,NaN,NaN,NaN,tr,NaN,NaN,mobile phone shops,259615,E_40c486192f6105,anker travel,37.844558,27.844448,adnan menderes bulvarı no 25,aydın,NaN,09000,tr,NaN,NaN,travel agencies,False
4,E_00001d92066153,restaurante casa cofiño,43.338196,-4.326821,NaN,caviedes,cantabria,NaN,es,NaN,NaN,spanish restaurants,504842,E_7e0d8e9138dd56,casa cofiño,43.338130,-4.326717,barrio de los caviedes sn,valdáliga cantabria,spain,39593,es,NaN,34942708046,spanish restaurants,True


In [13]:
t0 = datetime.datetime.now()
print('started @: {}'.format(t0))

df = df_raw.copy()[0:100000]
print(df.shape)

df = build_features(df)


print('time taken: {}'.format(datetime.datetime.now()-t0))

print()
print(df.shape)

df[0:20]

started @: 2022-07-06 09:43:52.907770
(100000, 26)
fuzzy match complete @ 2022-07-06 09:44:56.080967
point dist complete @ 2022-07-06 09:45:14.635043
same lang complete @ 2022-07-06 10:23:19.820232
len strings complete @ 2022-07-06 10:23:20.181051
time taken: 0:39:27.323262

(100000, 28)


,id_1,id_2,match,name_ratio,city_ratio,state_ratio,zip_ratio,country_ratio,url_ratio,categories_ratio,name_ratio_part,city_ratio_part,state_ratio_part,zip_ratio_part,country_ratio_part,url_ratio_part,categories_ratio_part,proximity,full_address_1_lang,full_address_2_lang,same_lang,len_name_diff,len_address_diff,len_city_diff,len_state_diff,len_zip_diff,len_phone_diff,len_categories_diff
0,E_000001272c6c5d,E_9e4a5adfe5b1c5,False,27.0,100.0,100.0,NaN,100.0,NaN,31.0,26.0,100.0,100.0,NaN,100.0,NaN,50.0,0.134442,nl,nl,1.0,-4.0,-3.0,0.0,0.0,NaN,NaN,-5.0
1,E_000002eae2a589,E_c12fb799a8a0e4,False,42.0,NaN,NaN,NaN,100.0,NaN,36.0,80.0,NaN,NaN,NaN,100.0,NaN,28.0,0.013043,it,pt,0.0,9.0,NaN,NaN,NaN,NaN,NaN,3.0
2,E_000007f24ebc95,E_b664578fe0e8c2,False,40.0,NaN,NaN,NaN,100.0,NaN,40.0,50.0,NaN,NaN,NaN,100.0,NaN,55.0,0.029438,th,th,1.0,8.0,NaN,NaN,NaN,NaN,NaN,8.0
3,E_000008a8ba4f48,E_40c486192f6105,False,40.0,NaN,NaN,NaN,100.0,NaN,18.0,57.0,NaN,NaN,NaN,100.0,NaN,27.0,0.022298,tr,tr,1.0,-4.0,-6.0,NaN,NaN,NaN,NaN,3.0
4,E_00001d92066153,E_7e0d8e9138dd56,True,65.0,21.0,29.0,NaN,100.0,NaN,100.0,100.0,38.0,40.0,NaN,100.0,NaN,100.0,0.011176,es,es,1.0,12.0,NaN,-12.0,4.0,NaN,NaN,0.0
5,E_00002a131a2bf6,E_cb119cbb1c6a97,False,7.0,NaN,NaN,NaN,100.0,NaN,22.0,8.0,NaN,NaN,NaN,100.0,NaN,20.0,0.128682,en,ar,0.0,5.0,NaN,NaN,NaN,NaN,NaN,7.0
6,E_0000764d65557e,E_f6468f03693270,False,17.0,NaN,NaN,NaN,100.0,NaN,11.0,22.0,NaN,NaN,NaN,100.0,NaN,29.0,0.011377,id,tl,0.0,-5.0,22.0,NaN,NaN,NaN,NaN,7.0
7,E_00007dcd2bb53f,E_f131dcb7f07be9,True,48.0,100.0,100.0,NaN,100.0,NaN,100.0,100.0,100.0,100.0,NaN,100.0,NaN,100.0,0.015663,en,en,1.0,11.0,NaN,0.0,0.0,NaN,NaN,0.0
8,E_0000890af22ff5,E_b041d061131959,False,30.0,NaN,NaN,NaN,100.0,NaN,41.0,44.0,NaN,NaN,NaN,100.0,NaN,50.0,0.046025,de,de,1.0,11.0,NaN,NaN,NaN,NaN,NaN,-5.0
9,E_0000c362229d93,E_d21943af626bf4,False,16.0,67.0,NaN,100.0,100.0,NaN,39.0,22.0,100.0,NaN,100.0,100.0,NaN,40.0,0.069098,en,en,1.0,-5.0,-4.0,5.0,NaN,0.0,NaN,0.0


In [15]:
df[df['full_address_2_lang'].isna()]

,id_1,id_2,match,name_ratio,city_ratio,state_ratio,zip_ratio,country_ratio,url_ratio,categories_ratio,name_ratio_part,city_ratio_part,state_ratio_part,zip_ratio_part,country_ratio_part,url_ratio_part,categories_ratio_part,proximity,full_address_1_lang,full_address_2_lang,same_lang,len_name_diff,len_address_diff,len_city_diff,len_state_diff,len_zip_diff,len_phone_diff,len_categories_diff
47275,E_0bbd93d358d31f,E_dbf14642cb38ef,False,3.0,NaN,NaN,NaN,100.0,NaN,NaN,8.0,NaN,NaN,NaN,100.0,NaN,NaN,0.188245,ca,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN
56204,E_0df6f216017c50,E_e80b88931cea68,True,0.0,NaN,NaN,NaN,100.0,NaN,100.0,0.0,NaN,NaN,NaN,100.0,NaN,100.0,0.010977,th,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0
